In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import sqlite3

In [6]:
imdb_path = r'C:\Users\espym.LAPTOP-41F90NSA\Desktop\zippedData\im.db\im.db'
conn = sqlite3.connect(imdb_path)


In [12]:
file_path1 = r'C:\Users\espym.LAPTOP-41F90NSA\projects\Movie_Project-1\tmdb.movies.csv.gz'
tmdbDF = pd.read_csv(file_path1)

In [13]:
filepath2 = r'C:\Users\espym.LAPTOP-41F90NSA\projects\Movie_Project-1\tn.movie_budgets.csv.gz'
tn_movieDF = pd.read_csv(filepath2)

In [14]:
filepath3 = r'C:\Users\espym.LAPTOP-41F90NSA\projects\Movie_Project-1\rt.movie_info.tsv.gz'
movie_infoDF = pd.read_csv(filepath3, delimiter='\t')

In [15]:
filepath4 =r'C:\Users\espym.LAPTOP-41F90NSA\projects\Movie_Project-1\rt.reviews.tsv.gz'
reviewsDF = pd.read_csv(filepath4, delimiter='\t', encoding='latin-1')

## Preview of Information in the data

Here we just want to have a look at whats contained in the IMDB dataset and just have a proper feel of whats contained in the dataset

In [29]:
#from the imdb database we first want to know the names and number of the tables.
query0 = ("""
    SELECT name
    AS 'Table_Name'
    FROM sqlite_master
    WHERE type = 'table'
;
""")
pd.read_sql(query0,conn)

,Table_Name
0,movie_basics
1,directors
2,known_for
3,movie_akas
4,movie_ratings
5,persons
6,principals
7,writers


In [32]:
# checking for the information contained in the movie basics table
query1 = ("""
    SELECT * 
    FROM movie_basics
    ;
          """)
pd.read_sql(query1,conn).head()

,movie_id,primary_title,original_title,start_year,runtime_minutes,genres
0,tt0063540,Sunghursh,Sunghursh,2013,175.0,"Action,Crime,Drama"
1,tt0066787,One Day Before the Rainy Season,Ashad Ka Ek Din,2019,114.0,"Biography,Drama"
2,tt0069049,The Other Side of the Wind,The Other Side of the Wind,2018,122.0,Drama
3,tt0069204,Sabse Bada Sukh,Sabse Bada Sukh,2018,NaN,"Comedy,Drama"
4,tt0100275,The Wandering Soap Opera,La Telenovela Errante,2017,80.0,"Comedy,Drama,Fantasy"


In [33]:
# In the reviews dataframe we will check for the last values
pd.read_sql(query1,conn).tail()

,movie_id,primary_title,original_title,start_year,runtime_minutes,genres
146139,tt9916538,Kuambil Lagi Hatiku,Kuambil Lagi Hatiku,2019,123.0,Drama
146140,tt9916622,Rodolpho Teóphilo - O Legado de um Pioneiro,Rodolpho Teóphilo - O Legado de um Pioneiro,2015,NaN,Documentary
146141,tt9916706,Dankyavar Danka,Dankyavar Danka,2013,NaN,Comedy
146142,tt9916730,6 Gunn,6 Gunn,2017,116.0,None
146143,tt9916754,Chico Albuquerque - Revelações,Chico Albuquerque - Revelações,2013,NaN,Documentary


In [35]:
#check the shape of the dataframe 
pd.read_sql(query1,conn).shape

(146144, 6)

In [37]:
#check for all the columns in the DF
pd.read_sql(query1,conn).columns

Index(['movie_id', 'primary_title', 'original_title', 'start_year',
       'runtime_minutes', 'genres'],
      dtype='object')

In [39]:
pd.read_sql(query1,conn).info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 146144 entries, 0 to 146143
Data columns (total 6 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   movie_id         146144 non-null  object 
 1   primary_title    146144 non-null  object 
 2   original_title   146123 non-null  object 
 3   start_year       146144 non-null  int64  
 4   runtime_minutes  114405 non-null  float64
 5   genres           140736 non-null  object 
dtypes: float64(1), int64(1), object(4)
memory usage: 6.7+ MB


In [40]:
pd.read_sql(query1,conn).describe()

,start_year,runtime_minutes
count,146144.000000,114405.000000
mean,2014.621798,86.187247
std,2.733583,166.360590
min,2010.000000,1.000000
25%,2012.000000,70.000000
50%,2015.000000,87.000000
75%,2017.000000,99.000000
max,2115.000000,51420.000000


In [47]:
# Checking for the 
df = pd.read_sql(query1,conn)
genre_column = df['genres']
df['genres'].unique

<bound method Series.unique of 0           Action,Crime,Drama
1              Biography,Drama
2                        Drama
3                 Comedy,Drama
4         Comedy,Drama,Fantasy
                  ...         
146139                   Drama
146140             Documentary
146141                  Comedy
146142                    None
146143             Documentary
Name: genres, Length: 146144, dtype: object>